<a href="https://colab.research.google.com/github/fedechaves/nw-bc/blob/main/day1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
os.listdir('/content/drive/MyDrive/nw_bc/')

['nw-version-reds.xlsx', 'Copia de nw-version-reds.xlsx', 'mats']

In [3]:
import pandas as pd

In [4]:
pd.__version__

'1.5.3'

In [26]:
df = pd.read_csv('/content/drive/MyDrive/nw_bc/mats/2023-nwbc-reds.csv')

In [27]:
df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,pitcher_name,batter,pitcher,events,description,...,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,attack_zone,swing,batter_name
0,FF,2022-10-05,93.8,2.39,5.44,"Hughes, Brandon",605244,676714,field_out,hit_into_play,...,2,15,Standard,Standard,133.0,0.0,-0.083,heart,0,"garcia, aramis"
1,FF,2022-10-05,93.3,2.30,5.51,"Hughes, Brandon",605244,676714,NaN,ball,...,2,15,Standard,Standard,132.0,0.0,0.014,shadow,0,"garcia, aramis"
2,SL,2022-10-05,84.3,2.45,5.21,"Hughes, Brandon",605244,676714,NaN,ball,...,2,15,Standard,Standard,216.0,0.0,0.011,waste,0,"garcia, aramis"
3,FF,2022-10-05,93.8,2.32,5.48,"Hughes, Brandon",605244,676714,NaN,called_strike,...,2,15,Standard,Standard,133.0,0.0,-0.023,heart,0,"garcia, aramis"
4,FF,2022-10-05,94.1,2.34,5.54,"Hughes, Brandon",605244,676714,NaN,swinging_strike,...,2,15,Standard,Standard,133.0,0.0,-0.017,heart,0,"garcia, aramis"


In [28]:
df.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'pitcher_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estim

In [29]:
df['home_score']

0        2
1        2
2        2
3        2
4        2
        ..
49033    0
49034    0
49035    0
49036    0
49037    0
Name: home_score, Length: 49038, dtype: int64

In [30]:
df['total_score'] = df['home_score'] + df['away_score']

In [31]:
df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,pitcher_name,batter,pitcher,events,description,...,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,attack_zone,swing,batter_name,total_score
0,FF,2022-10-05,93.8,2.39,5.44,"Hughes, Brandon",605244,676714,field_out,hit_into_play,...,15,Standard,Standard,133.0,0.0,-0.083,heart,0,"garcia, aramis",17
1,FF,2022-10-05,93.3,2.30,5.51,"Hughes, Brandon",605244,676714,NaN,ball,...,15,Standard,Standard,132.0,0.0,0.014,shadow,0,"garcia, aramis",17
2,SL,2022-10-05,84.3,2.45,5.21,"Hughes, Brandon",605244,676714,NaN,ball,...,15,Standard,Standard,216.0,0.0,0.011,waste,0,"garcia, aramis",17
3,FF,2022-10-05,93.8,2.32,5.48,"Hughes, Brandon",605244,676714,NaN,called_strike,...,15,Standard,Standard,133.0,0.0,-0.023,heart,0,"garcia, aramis",17
4,FF,2022-10-05,94.1,2.34,5.54,"Hughes, Brandon",605244,676714,NaN,swinging_strike,...,15,Standard,Standard,133.0,0.0,-0.017,heart,0,"garcia, aramis",17


In [32]:
df['description'].unique()

array(['hit_into_play', 'ball', 'called_strike', 'swinging_strike',
       'foul', 'blocked_ball', 'swinging_strike_blocked', 'hit_by_pitch',
       'foul_tip', 'foul_bunt', 'missed_bunt', 'bunt_foul_tip',
       'pitchout'], dtype=object)

In [33]:
strike_events = ['called_strike', 'swinging_strike',
       'foul', 'swinging_strike_blocked',
       'foul_tip', 'foul_bunt', 'missed_bunt', 'bunt_foul_tip'
       ]

In [34]:
strike_events

['called_strike',
 'swinging_strike',
 'foul',
 'swinging_strike_blocked',
 'foul_tip',
 'foul_bunt',
 'missed_bunt',
 'bunt_foul_tip']

In [35]:
#df['is_strike'] = df['description'].apply(lambda x: 1 if any(substring in x for substring in ['strike', 'foul', 'missed_bunt']) else 0)
#=IF(OR(ISNUMBER(FIND("strike", J2)), ISNUMBER(FIND("foul", J2)), ISNUMBER(FIND("missed_bunt", J2)))=TRUE, 1, 0)
#df['NewColumn'] = (df['J'].str.contains('strike|foul|missed_bunt')).astype(int)
df.loc[df['description'].isin(strike_events)]

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,pitcher_name,batter,pitcher,events,description,...,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,attack_zone,swing,batter_name,total_score
3,FF,2022-10-05,93.8,2.32,5.48,"Hughes, Brandon",605244,676714,NaN,called_strike,...,15,Standard,Standard,133.0,0.000,-0.023,heart,0,"garcia, aramis",17
4,FF,2022-10-05,94.1,2.34,5.54,"Hughes, Brandon",605244,676714,NaN,swinging_strike,...,15,Standard,Standard,133.0,0.000,-0.017,heart,0,"garcia, aramis",17
6,FF,2022-10-05,94.5,2.25,5.47,"Hughes, Brandon",608703,676714,NaN,called_strike,...,15,Standard,Standard,124.0,0.000,-0.024,shadow,0,"reynolds, matt",17
9,SI,2022-10-05,93.4,2.36,5.41,"Hughes, Brandon",641584,676714,NaN,foul,...,15,Standard,Standard,132.0,0.000,-0.066,chase,0,"fraley, jake",17
12,FF,2022-10-05,92.7,2.45,5.40,"Hughes, Brandon",641584,676714,NaN,called_strike,...,15,Standard,Standard,130.0,0.000,-0.034,heart,0,"fraley, jake",17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49033,NaN,2022-03-31,NaN,NaN,NaN,"Greene, Hunter",673357,668881,NaN,foul,...,0,NaN,NaN,NaN,0.000,NaN,waste,0,"robert, luis",0
49034,NaN,2022-03-31,NaN,NaN,NaN,"Greene, Hunter",673357,668881,NaN,swinging_strike,...,0,NaN,NaN,NaN,0.000,NaN,waste,0,"robert, luis",0
49035,NaN,2022-03-31,NaN,NaN,NaN,"Greene, Hunter",641313,668881,strikeout,foul_tip,...,0,NaN,NaN,NaN,0.022,NaN,waste,0,"anderson, tim",0
49036,NaN,2022-03-31,NaN,NaN,NaN,"Greene, Hunter",641313,668881,NaN,called_strike,...,0,NaN,NaN,NaN,0.000,NaN,waste,0,"anderson, tim",0


In [36]:
df['is_strike'] = 0
df.loc[df['description'].isin(strike_events), 'is_strike'] = 1

In [37]:
df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,pitcher_name,batter,pitcher,events,description,...,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,attack_zone,swing,batter_name,total_score,is_strike
0,FF,2022-10-05,93.8,2.39,5.44,"Hughes, Brandon",605244,676714,field_out,hit_into_play,...,Standard,Standard,133.0,0.0,-0.083,heart,0,"garcia, aramis",17,0
1,FF,2022-10-05,93.3,2.30,5.51,"Hughes, Brandon",605244,676714,NaN,ball,...,Standard,Standard,132.0,0.0,0.014,shadow,0,"garcia, aramis",17,0
2,SL,2022-10-05,84.3,2.45,5.21,"Hughes, Brandon",605244,676714,NaN,ball,...,Standard,Standard,216.0,0.0,0.011,waste,0,"garcia, aramis",17,0
3,FF,2022-10-05,93.8,2.32,5.48,"Hughes, Brandon",605244,676714,NaN,called_strike,...,Standard,Standard,133.0,0.0,-0.023,heart,0,"garcia, aramis",17,1
4,FF,2022-10-05,94.1,2.34,5.54,"Hughes, Brandon",605244,676714,NaN,swinging_strike,...,Standard,Standard,133.0,0.0,-0.017,heart,0,"garcia, aramis",17,1


#homework assignment
1. use the strikes column to find whre there are 2 strikes and make a new column indicating wether the pitch was thrown in 2 strikes situation. name this 'is_2_strikes'

2. use the 'release_speed' column to find when a pitch is thrown greather than 94 mph (the column is in mph already) name this `is_high_velo`

3. use the description column  AND the attack_zone column to indicate when a pitch is thrown for a called or swinging strike (not a foul) in the "heart" attack zone (hint: this requires two conditional statements! maybe google or ask for help in discord)

In [38]:
df['strikes']

0        2
1        2
2        2
3        1
4        0
        ..
49033    1
49034    0
49035    2
49036    1
49037    0
Name: strikes, Length: 49038, dtype: int64

In [39]:
df['is_2_strikes'] = 0
df.loc[df['strikes'] >= 2, 'is_2_strikes'] = 1

In [41]:
df['is_2_strikes']

0        1
1        1
2        1
3        0
4        0
        ..
49033    0
49034    0
49035    1
49036    0
49037    0
Name: is_2_strikes, Length: 49038, dtype: int64

In [42]:
df['is_high_velo'] = 0
df.loc[df['release_speed'] > 94, 'is_high_velo'] = 1

In [47]:
df['description'].unique()


array(['hit_into_play', 'ball', 'called_strike', 'swinging_strike',
       'foul', 'blocked_ball', 'swinging_strike_blocked', 'hit_by_pitch',
       'foul_tip', 'foul_bunt', 'missed_bunt', 'bunt_foul_tip',
       'pitchout'], dtype=object)

In [54]:
df['pitch_called'] = 0
df.loc[(df['description'].isin(['called_strike','swinging_strike'])) & (df['attack_zone'] == 'heart'), 'pitch_called'] = 1

In [55]:
df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,pitcher_name,batter,pitcher,events,description,...,delta_home_win_exp,delta_run_exp,attack_zone,swing,batter_name,total_score,is_strike,is_2_strikes,is_high_velo,pitch_called
0,FF,2022-10-05,93.8,2.39,5.44,"Hughes, Brandon",605244,676714,field_out,hit_into_play,...,0.0,-0.083,heart,0,"garcia, aramis",17,0,1,0,0
1,FF,2022-10-05,93.3,2.30,5.51,"Hughes, Brandon",605244,676714,NaN,ball,...,0.0,0.014,shadow,0,"garcia, aramis",17,0,1,0,0
2,SL,2022-10-05,84.3,2.45,5.21,"Hughes, Brandon",605244,676714,NaN,ball,...,0.0,0.011,waste,0,"garcia, aramis",17,0,1,0,0
3,FF,2022-10-05,93.8,2.32,5.48,"Hughes, Brandon",605244,676714,NaN,called_strike,...,0.0,-0.023,heart,0,"garcia, aramis",17,1,0,0,1
4,FF,2022-10-05,94.1,2.34,5.54,"Hughes, Brandon",605244,676714,NaN,swinging_strike,...,0.0,-0.017,heart,0,"garcia, aramis",17,1,0,1,1
